# **Generación del dataset de Nivel 1**

En este notebook generamos el dataser de las preguntas/respuestas de Nivel 1. Hemos dividido la generación en 3 subdatasets:

- Preguntas y respuestas de un dato en una columna determinada
- Preguntas y respuestas de dos datos en columnas determinadas
- Preguntas y respuestas de un dato de cualquier columna

Cargamos el dataset, y hacemos un repaso rápido.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import random
import itertools

In [ ]:
import json
from typing import Dict, List, Optional, Tuple

In [ ]:
strings = {'Sección' : 'str', 'cod_ccaa' : 'str', 'cod_prov' : 'str', 'cod_mun' : 'str', 'cod_sec' : 'str'}

In [ ]:
df_eleccion = pd.read_csv('/content/drive/MyDrive/Practica_LLM_Engineering_25/df_elecciones_19.csv', dtype = strings)

In [ ]:
df_eleccion.head()

,Elecciones,Sección,cod_ccaa,cod_prov,cod_mun,cod_sec,CCAA,Provincia,Municipio,Distrito,...,Renta persona 2017,Renta persona 2015,Renta hogar 2017,Renta hogar 2015,Renta Salarios 2018,Renta Salarios 2015,Renta Pensiones 2018,Renta Pensiones 2015,Renta Desempleo 2018,Renta Desempleo 2015
0,Abril 2019,022019041010400101001,01,04,04001,0400101001,Andalucía,Almería,Abla,1,...,9159.0,8788.0,20172.0,19546.0,5574.0,4833.0,3286.0,3082.0,403.0,471.0
1,Abril 2019,022019041010400201001,01,04,04002,0400201001,Andalucía,Almería,Abrucena,1,...,8827.0,8107.0,17841.0,17115.0,4640.0,4048.0,3418.0,2770.0,568.0,620.0
2,Abril 2019,022019041010400301001,01,04,04003,0400301001,Andalucía,Almería,Adra,1,...,8965.0,8267.0,26498.0,24688.0,5121.0,4795.0,2499.0,2301.0,337.0,333.0
3,Abril 2019,022019041010400301002,01,04,04003,0400301002,Andalucía,Almería,Adra,1,...,8599.0,7941.0,25677.0,23400.0,5381.0,4837.0,1815.0,1724.0,343.0,464.0
4,Abril 2019,022019041010400301003,01,04,04003,0400301003,Andalucía,Almería,Adra,1,...,8076.0,7150.0,22051.0,19687.0,5224.0,4044.0,1170.0,1198.0,416.0,476.0


In [ ]:
df_eleccion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72619 entries, 0 to 72618
Data columns (total 97 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Elecciones                        72619 non-null  object 
 1   Sección                           72619 non-null  object 
 2   cod_ccaa                          72619 non-null  object 
 3   cod_prov                          72619 non-null  object 
 4   cod_mun                           72619 non-null  object 
 5   cod_sec                           72619 non-null  object 
 6   CCAA                              72619 non-null  object 
 7   Provincia                         72619 non-null  object 
 8   Municipio                         72619 non-null  object 
 9   Distrito                          72619 non-null  int64  
 10  Censo_Esc                         72619 non-null  int64  
 11  Votos_Total                       72619 non-null  int64  
 12  Part

Estos son los templates de las preguntas en el primer caso. Vemos que preguntamos siempre datos de una determinada sección en una elección.

In [ ]:
templates_nivel1 = [
    # Template: {partido}, {municipio}, {año}, {valor}
    "¿Qué porcentaje de votos obtuvo IU en la sección {seccion} en las elecciones de {eleccion}?",

    "¿Qué porcentaje de votos obtuvo el PP en la sección {seccion} en las elecciones de {eleccion}?",

    # Template: {municipio}, {año}, {partido_ganador}
    "¿Qué partido ganó las elecciones en {seccion} en {eleccion}?",

    # Template: {municipio}, {año}, {valor}
    "¿Cuál fue la participación electoral en la sección {seccion} en las elecciones de {eleccion}?",

    # Template: {municipio}, {año}, {comparación}
    "¿Cuántos votos válidos hubo en la sección {seccion} en las elecciones de {eleccion}?",

    "¿A qué municipio pertenece la seccion {seccion} en la elección {eleccion}?",

    "¿A qué CCAA pertenece la seccion {seccion} en la elección {eleccion}?",

    "¿Qué porcentaje de afiliados de SS hubo en la seccion {seccion} en las elecciones de {eleccion}?",

    "¿Cuál fue la renta de salarios 2018 en la sección {seccion} en las elecciones de {eleccion}?",

    "¿Cuál fue la renta de desempleo 2018 en la sección {seccion} en las elecciones de {eleccion} y qué partido resultó vencedor?",

    "¿Qué partido quedó en segundo lugar en la seccion {seccion} en las elecciones de {eleccion} y qué porcentaje obtuvo?",

    "¿Qué partido quedó en tercer lugar en la seccion {seccion} en las elecciones de {eleccion} y qué renta pensiones 2018 había?",

    "¿Qué partido quedó en cuarto lugar en la seccion {seccion} en las elecciones de {eleccion}?",

    "¿Cuántos votos consiguió el PP en la seccion {seccion} en las elecciones de {eleccion}?"
]

Estos son los templates de las respuestas

In [ ]:
respuestas_nivel1 = [
  "El partido IU obtuvo en la sección {seccion} el {result_value:.2f}% de votos en las eleccione de {eleccion}.",
  "El partido PP obtuvo en la sección {seccion} el {result_value:.2f}% de votos en las eleccione de {eleccion}.",
  "El las elecciones de {eleccion}, el ganador en la sección {seccion} fue el partido {result_value}.",
  "La participación electoral en la sección {seccion} en las elecciones de {eleccion} la participación fue del {result_value:.2f}%.",
  "En la sección {seccion} en las elecciones de {eleccion} hubo {result_value} votos válidos.",
  "La sección {seccion} en las elecciones de {eleccion} pertenece al municipio {result_value}.",
  "La sección {seccion} en las elecciones de {eleccion} pertenece a la CCAA {result_value}.",
  "El porcentaje de afiliados de SS en la sección {seccion} en las elecciones de {eleccion} fue del {result_value:.2f}%.",
  "La renta de salarios 2018 en la sección {seccion} en las elecciones de {eleccion} fue de {result_value}.",
  "La renta de desempleo 2018 en la sección {seccion} en las elecciones de {eleccion} fue de {result_value}.",
  "El segundo lugar en la sección {seccion} en las elecciones de {eleccion} fue el partido {result_value}.",
  "El tercer lugar en la sección {seccion} en las elecciones de {eleccion} fue el partido {result_value}.",
  "El cuarto lugar en la sección {seccion} en las elecciones de {eleccion} fue el partido {result_value}.",
  "El PP en la sección {seccion} en las elecciones de {eleccion} obtuvo {result_value} votos."
]

Esta es una lista que nos sirve para la búsqueda de las columnas necesarias.

In [ ]:
columnas = [

  [['cod_sec', 'Elecciones'], ['% IU']],
  [['cod_sec', 'Elecciones'], ['% PP']],
  [['cod_sec', 'Elecciones'], ['Ganador']],
  [['cod_sec', 'Elecciones'], ['Participación']],
  [['cod_sec', 'Elecciones'], ['Votos_Válidos']],
  [['cod_sec', 'Elecciones'], ['Municipio']],
  [['cod_sec', 'Elecciones'], ['CCAA']],
  [['cod_sec', 'Elecciones'], ['% Afiliados SS / Población']],
  [['cod_sec', 'Elecciones'], ['Renta Salarios 2018']],

  [['cod_sec', 'Elecciones'], ['Renta Desempleo 2018']],
  [['cod_sec', 'Elecciones'], ['Segundo']],
  [['cod_sec', 'Elecciones'], ['Tercero']],

  [['cod_sec', 'Elecciones'], ['Cuarto']],
  [['cod_sec', 'Elecciones'], ['PP']]

]

Esta es la función que genera el primer subdataset, seleccionando una sección y elección de forma aleatoria, junto a el template elegido de la misma manera.

In [ ]:
def generar_pregunta_respuesta_nivel1(df, templates, respuestas, columnas):

  index = random.randint(0, len(templates)-1)

  template = templates[index]
  respuesta_template_str = respuestas[index]
  columna_name = columnas[index][1][0]

  df_working = df.copy()

  # Keep trying until a valid seccion and eleccion pair is found
  while True:
      seccion_options = df_working['cod_sec'].unique()
      if len(seccion_options) == 0:
          raise ValueError("No unique 'cod_sec' found in the DataFrame.")

      seccion = random.choice(seccion_options)

      # Filter by the chosen seccion
      df_filtered_seccion = df_working[df_working['cod_sec'] == seccion]

      if not df_filtered_seccion.empty:
          eleccion_options = df_filtered_seccion['Elecciones'].unique()
          if len(eleccion_options) > 0:
              eleccion = random.choice(eleccion_options)
              break # Found a valid seccion and eleccion, exit loop

  df_filtered_seccion = df_filtered_seccion[df_filtered_seccion['Elecciones'] == eleccion]

  pregunta = template.format(seccion = seccion, eleccion = eleccion)
  # print(pregunta)

  otro = df_filtered_seccion[columna_name].values[0]

  # Determine if 'otro' needs to be multiplied by 100 for display
  percentage_columns = ['% IU', '% PP', 'Participación', '% Afiliados SS / Población']
  if columna_name in percentage_columns:
      result_value = otro * 100
  else:
      result_value = otro

  # For formatting integers like Votos_Validos or PP votes, ensure they are not floats.
  # This is a refinement for display purposes.
  if isinstance(result_value, float) and columna_name in ['Votos_Válidos', 'PP']:
      result_value = int(result_value)

  respuesta = respuesta_template_str.format(seccion = seccion, eleccion = eleccion, result_value = result_value)
  # print(result_value)
  # print(respuesta)

  #return result_value

  return {
        "instruction": pregunta,
        "input": f"Datos de {seccion} en {eleccion}: {len(df_filtered_seccion[columna_name])} secciones electorales",
        "output": respuesta,
        "metadata": {
            "tipo": "nivel1 - Respuesta única",
            "Sección": seccion,
            "Elección": eleccion,
            "datos_reales": True
        }
    }


Este es un ejemplo de la generación de preguntas/respuestas, que se producen en formato diccionario.

In [ ]:
generar_pregunta_respuesta_nivel1(df_eleccion, templates_nivel1, respuestas_nivel1, columnas)

¿Qué porcentaje de votos obtuvo el PP en la sección 2807921012 en las elecciones de Abril 2019?
20.379146919431278
El partido PP obtuvo en la sección 2807921012 el 20.38% de votos en las eleccione de Abril 2019


{'instruction': '¿Qué porcentaje de votos obtuvo el PP en la sección 2807921012 en las elecciones de Abril 2019?',
 'input': 'Datos de 2807921012 en Abril 2019: 1 secciones electorales',
 'output': 'El partido PP obtuvo en la sección 2807921012 el 20.38% de votos en las eleccione de Abril 2019',
 'metadata': {'tipo': 'nivel1 - Respuesta única',
  'municipio': '2807921012',
  'año': 'Abril 2019',
  'datos_reales': True}}

El siguiente subdataset contempla dos datos en vez de uno, pero el proceso es realmente el mismo. Consideramos, de todas formas, menos posibles templates.

In [ ]:
templates_nivel1_2 = [

'¿A qué provincia pertenece la sección {seccion} en la elección {eleccion} y cuál es su renta salarios 2018?',

'¿Cúal fue la participación en la sección {seccion} en la elección {eleccion} y que porcentaje obtuvo el PP?',

'¿Cuál es el tamaño del censo y la población total de la sección {seccion} en la elección {eleccion}?',

'¿Cuál fue el porcentaje de mayores de 65 años en la sección {seccion} en la elección {eleccion} y cuál fue la renta hogar 2017?',

'¿Qué partido quedó en quinto lugar en la sección {seccion} en la elección de {eleccion} y que porcentaje de voto obtuvo Vox?'

]

In [ ]:
respuestas_nivel1_2 = [

    'La sección {seccion} en la elección {eleccion} pertenece a la provincia {result_value} y su renta de salarios 2018 es de {result_value_2}.',

    'La participación en la sección {seccion} en la elección {eleccion} fue del {result_value:.2f}% y el PP obtuvo un porcentaje del {result_value_2:.2f}%.',

    'El censo en la sección {seccion} en la elección de {eleccion} fue de {result_value} habitantes y la población total fue de {result_value_2}.',

    'El porcentaje de mayores de 65 años en la sección {seccion} en la eleccion de {eleccion} fue del {result_value:.2f}% y la renta hogar 2017 fue de {result_value_2}.',

    'El quinto lugar en la sección {seccion} en la elección de {eleccion} fue el partido {result_value} y su porcentaje de voto de Vox fue el {result_value_2:.2f}%.'

]




In [ ]:
columnas_2 = [

  [['cod_sec', 'Elecciones'], ['Provincia','Renta Salarios 2018']],
  [['cod_sec', 'Elecciones'], ['Participación', '% PP']],
  [['cod_sec', 'Elecciones'], ['Censo_Esc', 'Población Total']],
  [['cod_sec', 'Elecciones'], ['% mayores 65 años', 'Renta hogar 2017']],
  [['cod_sec', 'Elecciones'], ['Quinto', '% VOX']]

]

In [ ]:
def generar_pregunta_respuesta_nivel1_2(df, templates, respuestas, columnas):
  index = random.randint(0, len(templates)-1)

  template = templates[index]
  respuesta_template_str = respuestas[index]
  columna_names_list = columnas[index][1]

  df_working = df.copy()

  # Keep trying until a valid seccion and eleccion pair is found
  while True:
      seccion_options = df_working['cod_sec'].unique()
      if len(seccion_options) == 0:
          raise ValueError("No unique 'cod_sec' found in the DataFrame.")

      seccion = random.choice(seccion_options)

      # Filter by the chosen seccion
      df_filtered_seccion = df_working[df_working['cod_sec'] == seccion]

      if not df_filtered_seccion.empty:
          eleccion_options = df_filtered_seccion['Elecciones'].unique()
          if len(eleccion_options) > 0:
              eleccion = random.choice(eleccion_options)
              break # Found a valid seccion and eleccion, exit loop

  df_filtered_seccion = df_filtered_seccion[df_filtered_seccion['Elecciones'] == eleccion]

  pregunta = template.format(seccion = seccion, eleccion = eleccion)
  # print(pregunta)

  otro_1 = df_filtered_seccion[columna_names_list[0]].values[0]

  # Determine if 'otro_1' needs to be multiplied by 100 for display
  percentage_columns = ['% VOX', '% PP', 'Participación', '% mayores 65 años']
  if columna_names_list[0] in percentage_columns:
      result_value = otro_1 * 100
  else:
      result_value = otro_1

  # For formatting integers like Votos_Validos or PP votes, ensure they are not floats.
  # This is a refinement for display purposes.
  if isinstance(result_value, float) and columna_names_list[0] in ['Censo_Esc', 'Población Total']:
      result_value = int(result_value)


  otro_2 = df_filtered_seccion[columna_names_list[1]].values[0]

  # Determine if 'otro_2' needs to be multiplied by 100 for display
  percentage_columns = ['% VOX', '% PP', 'Participación', '% mayores 65 años']
  if columna_names_list[1] in percentage_columns:
      result_value_2 = otro_2 * 100
  else:
      result_value_2 = otro_2

  # For formatting integers like Votos_Validos or PP votes, ensure they are not floats.
  # This is a refinement for display purposes.
  if isinstance(result_value, float) and columna_names_list[1] in ['Censo_Esc', 'Población Total']:
      result_value_2 = int(result_value)

  respuesta = respuesta_template_str.format(seccion = seccion, eleccion = eleccion, result_value = result_value, result_value_2 = result_value_2)
  # print(result_value)
  # print(respuesta)

  return {
        "instruction": pregunta,
        "input": f"Datos de {seccion} en {eleccion}: {len(df_filtered_seccion)} secciones electorales",
        "output": respuesta,
        "metadata": {
            "tipo": "nivel1 - Respuesta doble",
            "Sección": seccion,
            "Elección": eleccion,
            "datos_reales": True
        }
    }


In [ ]:
generar_pregunta_respuesta_nivel1_2(df_eleccion, templates_nivel1_2, respuestas_nivel1_2, columnas_2)

¿Cuál es el tamaño del censo y la población total de la sección 0401306021 en la elección Abril 2019?
1295
El censo en la sección 0401306021 en la elección de Abril 2019 fue de 1295 habitantes y la población total fue de 1917.0.


{'instruction': '¿Cuál es el tamaño del censo y la población total de la sección 0401306021 en la elección Abril 2019?',
 'input': 'Datos de 0401306021 en Abril 2019: 1 secciones electorales',
 'output': 'El censo en la sección 0401306021 en la elección de Abril 2019 fue de 1295 habitantes y la población total fue de 1917.0.',
 'metadata': {'tipo': 'nivel1 - Respuesta doble',
  'municipio': '0401306021',
  'año': 'Abril 2019',
  'datos_reales': True}}

Finalmente, consideramos el terder y último subdataset, que es la búsqueda de un dato en cualquier columna para una elección y sección concreta.

In [ ]:
templates_nivel1_3 = [

'¿Cuál es el resultado de la columna {col} en la sección {seccion} en la elección {eleccion}?',

]

In [ ]:
respuestas_nivel1_3 = [

    'El resultado de la columna {col} de la sección {seccion} en la elección {eleccion} es {result_value}'
]

Excluimos, naturalmente, las columnas de sección y elección.

In [ ]:
columnas_3 = df_eleccion.columns.tolist()
columnas_3.remove('cod_sec')
columnas_3.remove('Elecciones')

In [ ]:
columnas_3

['Sección',
 'cod_ccaa',
 'cod_prov',
 'cod_mun',
 'CCAA',
 'Provincia',
 'Municipio',
 'Distrito',
 'Censo_Esc',
 'Votos_Total',
 'Participación',
 'Nulos',
 'Votos_Válidos',
 'Blanco',
 'V_Cand',
 'PP',
 'PSOE',
 'Cs',
 'UP',
 'IU',
 'VOX',
 'UPyD',
 'MP',
 'CiU',
 'ERC',
 'JxC',
 'CUP',
 'DiL',
 'PNV',
 'Bildu',
 'Amaiur',
 'CC',
 'FA',
 'TE',
 'BNG',
 'PRC',
 'GBai',
 'Compromis',
 'PACMA',
 'Otros',
 '% PP',
 '% PSOE',
 '% UP',
 '% VOX',
 '% Cs',
 '% IU',
 'Ganador',
 'Segundo',
 'Tercero',
 'Cuarto',
 'Quinto',
 'edad_0-4',
 'edad_5-9',
 'edad_10-14',
 'edad_15-19',
 'edad_20-24',
 'edad_25-29',
 'edad_30-34',
 'edad_35-39',
 'edad_40-44',
 'edad_45-49',
 'edad_50-54',
 'edad_55-59',
 'edad_60-64',
 'edad_65-69',
 'edad_70-74',
 'edad_75-79',
 'edad_80-84',
 'edad_85-89',
 'edad_90-94',
 'edad_95-99',
 'edad_100 y más',
 'Población Total',
 'Hombres',
 'Mujeres',
 '% mayores 65 años',
 '% 20-64 años',
 '% menores 19 años',
 'Afiliados SS Minicipio',
 '% Afiliados SS autónomos',
 

In [ ]:
def generar_pregunta_respuesta_nivel1_3(df, templates, respuestas, columnas):


  # index = random.randint(0, len(templates)-1)

  template = templates[0]
  respuesta_template_str = respuestas[0]
  columna_name = random.choice(columnas)

  df_working = df.copy()

  # Keep trying until a valid seccion and eleccion pair is found
  while True:
      seccion_options = df_working['cod_sec'].unique()
      if len(seccion_options) == 0:
          raise ValueError("No unique 'cod_sec' found in the DataFrame.")

      seccion = random.choice(seccion_options)

      # Filter by the chosen seccion
      df_filtered_seccion = df_working[df_working['cod_sec'] == seccion]

      if not df_filtered_seccion.empty:
          eleccion_options = df_filtered_seccion['Elecciones'].unique()
          if len(eleccion_options) > 0:
              eleccion = random.choice(eleccion_options)
              break # Found a valid seccion and eleccion, exit loop

  df_filtered_seccion = df_filtered_seccion[df_filtered_seccion['Elecciones'] == eleccion]

  pregunta = template.format(seccion = seccion, eleccion = eleccion, col = columna_name)
  # print(pregunta)

  result_value = df_filtered_seccion[columna_name].values[0]

  respuesta = respuesta_template_str.format(seccion = seccion, eleccion = eleccion, result_value = result_value, col = columna_name)
  # print(result_value)
  # print(respuesta)

  return {
        "instruction": pregunta,
        "input": f"Datos de {seccion} en {eleccion}: {len(df_filtered_seccion)} secciones electorales",
        "output": respuesta,
        "metadata": {
            "tipo": "nivel1 - Columna aleatoria",
            "Sección": seccion,
            "Elección": eleccion,
            "datos_reales": True
        }
    }

In [ ]:
generar_pregunta_respuesta_nivel1_3(df_eleccion, templates_nivel1_3, respuestas_nivel1_3, columnas_3)

¿Cuál es el resultado de la columna Ganador en la sección 0807303003 en la elección Noviembre 2019?
PSOE
El resultado de la columna Ganador de la sección 0807303003 en la elección Noviembre 2019 es PSOE


{'instruction': '¿Cuál es el resultado de la columna Ganador en la sección 0807303003 en la elección Noviembre 2019?',
 'input': 'Datos de 0807303003 en Noviembre 2019: 1 secciones electorales',
 'output': 'El resultado de la columna Ganador de la sección 0807303003 en la elección Noviembre 2019 es PSOE',
 'metadata': {'tipo': 'nivel1 - Columna aleatoria',
  'municipio': '0807303003',
  'año': 'Noviembre 2019',
  'datos_reales': True}}

Esta es la función para generar el dataset del Nivel 1, con 2.000 preguntas y respuestas. El peso entre subdatasets hemos asumido que es 45/20/35%

In [ ]:
def generar_dataset_nivel1(df, n_ejemplos: int = 2000, output_path: str = "dataset_nivel1.jsonl"):
    """
    Genera un dataset completo de preguntas nivel 2

    Args:
        df_path: Ruta al archivo CSV
        n_ejemplos: Número de ejemplos a generar
        output_path: Ruta donde guardar el dataset
    """
    # Cargar datos
    # print(f"Cargando datos desde {df_path}...")

    # strings = {'Sección' : 'str', 'cod_ccaa' : 'str', 'cod_prov' : 'str', 'cod_mun' : 'str', 'cod_sec' : 'str'}


    # df = pd.read_csv(df_path, dtype = strings)

    # Si tienes múltiples datasets, añadir columna 'Año'
    # Ejemplo: df['Año'] = 2019  # O leer de nombre de archivo

    # Inicializar generador
    # generador = GeneradorPreguntasNivel2(df)

    # Generar ejemplos
    dataset = []
    print(f"Generando {n_ejemplos} ejemplos nivel 1...")

    choice_funcion = ['0', '1', '2']

    pesos = [0.45, 0.20, 0.35]

    summary_choices = {'0' : 0, '1' : 1, '2' : 0}

    for i in range(n_ejemplos):
        if i % 100 == 0:
            print(f"  Progreso: {i}/{n_ejemplos}")

        choice = random.choices(choice_funcion, weights = pesos, k = 1)[0]

        # print(choice)

        if choice == '0':
          ejemplo = generar_pregunta_respuesta_nivel1(df, templates_nivel1, respuestas_nivel1, columnas)
          summary_choices['0'] += 1

        elif choice == '1':
          ejemplo = generar_pregunta_respuesta_nivel1_2(df, templates_nivel1_2, respuestas_nivel1_2, columnas_2)
          summary_choices['1'] += 1

        elif choice == '2':
          ejemplo = generar_pregunta_respuesta_nivel1_3(df, templates_nivel1_3, respuestas_nivel1_3, columnas_3)
          summary_choices['2'] += 1


        # ejemplo = generador.generar_ejemplo_nivel2()
        if ejemplo is not None:
            dataset.append(ejemplo)

    print(f"Generados {len(dataset)} ejemplos válidos.")


    # Estadísticas por tipo
    # tipos = [e['metadata']['tipo'] for e in dataset]
    print("\nDistribución por tipo:")
    for tipo in summary_choices.keys():
        # count = tipos.count(tipo)
        print(f"Ejemplos del tipo {tipo}: {summary_choices[tipo]} ({summary_choices[tipo]/len(dataset)*100:.1f}%)")



    # Guardar en formato JSONL (una línea por ejemplo)
    print(f"\nGuardando en {output_path}...")
    with open(output_path, 'w', encoding='utf-8') as f:
        for ejemplo in dataset:
            f.write(json.dumps(ejemplo, ensure_ascii=False) + '\n')

    # También guardar versión para inspección
    with open(output_path.replace('.jsonl', '_pretty.json'), 'w', encoding='utf-8') as f:
        json.dump(dataset, f, ensure_ascii=False, indent=2)

    print("¡Dataset generado exitosamente!")
    return dataset

Ejecutamos la generación del dataset, y lo grabamos en Drive. La función nos da el peso de cada subdataset, que está en línea con lo que habíamos previsto.

In [ ]:

# ============================================================================
# EJEMPLO DE USO
# ============================================================================

if __name__ == "__main__":
    # Configuración
    # RUTA_CSV = "/content/drive/MyDrive/Practica_LLM_Engineering_25/df_elecciones_19.csv"  # Tu archivo
    NUM_EJEMPLOS = 2000  # Para nivel 1
    RUTA_SALIDA = "/content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_preguntas_nivel1.jsonl"

    # Ejecutar generación
    dataset = generar_dataset_nivel1(df = df_eleccion,
        n_ejemplos = NUM_EJEMPLOS,
        output_path = RUTA_SALIDA
    )

    # Mostrar algunos ejemplos
    print("\n\n--- EJEMPLOS GENERADOS ---")
    for i, ejemplo in enumerate(dataset[:3]):
        print(f"\nEjemplo {i+1}:")
        print(f"Pregunta: {ejemplo['instruction']}")
        print(f"Respuesta: {ejemplo['output'][:150]}...")
        print(f"Tipo: {ejemplo['metadata']['tipo']}")



Generando 2000 ejemplos nivel 1...
  Progreso: 0/2000
  Progreso: 100/2000
  Progreso: 200/2000
  Progreso: 300/2000
  Progreso: 400/2000
  Progreso: 500/2000
  Progreso: 600/2000
  Progreso: 700/2000
  Progreso: 800/2000
  Progreso: 900/2000
  Progreso: 1000/2000
  Progreso: 1100/2000
  Progreso: 1200/2000
  Progreso: 1300/2000
  Progreso: 1400/2000
  Progreso: 1500/2000
  Progreso: 1600/2000
  Progreso: 1700/2000
  Progreso: 1800/2000
  Progreso: 1900/2000
Generados 2000 ejemplos válidos.

Distribución por tipo:
Ejemplos del tipo 0: 941 (47.0%)
Ejemplos del tipo 1: 362 (18.1%)
Ejemplos del tipo 2: 698 (34.9%)

Guardando en /content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_preguntas_nivel1.jsonl...
¡Dataset generado exitosamente!


--- EJEMPLOS GENERADOS ---

Ejemplo 1:
Pregunta: ¿Cuál es el resultado de la columna Paro Registrado Municipio en la sección 2305004015 en la elección Noviembre 2019?
Respuesta: El resultado de la columna Paro Registrado Municipio de la sección 230